In [4]:
# !python -m spacy init fill-config  config.cfgbase_config.cfg

In [2]:
# ! python -m spacy debug data base_config.cfg

✘ Config validation error
disabled	field required
tokenizer	field required
before_creation	field required
after_creation	field required
after_pipeline_creation	field required
{'lang': 'en', 'pipeline': ['transformer', 'ner'], 'batch_size': 128, 'vectors': {'@vectors': 'spacy.Vectors.v1'}}

If your config contains missing values, you can run the 'init fill-config'
command to fill in all the defaults, if possible:

python -m spacy init fill-config base_config.cfg base_config.cfg 



In [29]:
!python -m spacy init config config.cfg --lang en --pipeline ner -F

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [16]:
from datasets import load_dataset
import spacy
from spacy.tokens import DocBin
from datasets import load_dataset

In [13]:

dataset = load_dataset("eriktks/conll2003",trust_remote_code=True)
train_data = dataset['train']


In [22]:
from datasets import load_dataset
import spacy
from spacy.tokens import DocBin
from datasets import Dataset



dataset = load_dataset("eriktks/conll2003", trust_remote_code=True)
label_names = dataset["train"].features["ner_tags"].feature.names

nlp = spacy.blank("en")

def convert_to_docbin_simple(dataset: Dataset, label_names: list, output_path: str):
    nlp = spacy.blank("en")  # just tokenizer
    doc_bin = DocBin()

    for example in dataset:
        words = example["tokens"]
        tags = example["ner_tags"]
        doc = nlp.make_doc(" ".join(words))

        ents = []
        offset = 0
        for word, tag_id in zip(words, tags):
            tag = label_names[tag_id]
            if tag.startswith("B-"):
                start = doc.text.find(word, offset)
                end = start + len(word)
                label = tag[2:]
                span = doc.char_span(start, end, label=label)
                if span:
                    ents.append(span)
                offset = end
            else:
                offset = doc.text.find(word, offset) + len(word)

        doc.ents = ents
        doc_bin.add(doc)

    doc_bin.to_disk(output_path)
    print(f"Saved to {output_path}")

convert_to_docbin_simple(dataset["train"], label_names, "train.spacy")
convert_to_docbin_simple(dataset["validation"], label_names, "dev.spacy")

Saved to train.spacy
Saved to dev.spacy


In [31]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy --gpu-id 0


ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     44.28    0.00    0.00    0.00    0.00
  0     200         64.88   2641.24   59.39   60.24   58.57    0.59
  0     400        159.14   1584.51   75.21   76.36   74.10    0.75
  0     600        178.19   1502.86   79.33   79.77   78.90    0.79
  0     800        211.72   1573.32   82.89   83.75   82.04    0.83
  0    1000        553.33   1653.04   83.41   84.03   82.80    0.83
  1    1200        261.85   1571.59   86.02   86.86   85.19    0.86
  1    1400        295.69   1136.86   86.41   86.44   86.39    0.86
  1    1600        415.07   

In [32]:
!python -m spacy evaluate ./output/model-best ./dev.spacy


ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

================================== Results ==================================

TOK     100.00
NER P   88.94 
NER R   89.33 
NER F   89.14 
SPEED   27796 


=============================== NER (per type) ===============================

           P       R       F
ORG    87.58   82.55   84.99
LOC    89.65   94.34   91.94
MISC   89.44   85.47   87.41
PER    88.89   91.21   90.03



In [33]:

# Load best model after training
nlp = spacy.load("./output/model-best")

# Use it
doc = nlp("Apple is looking at buying a startup in the UK")
for ent in doc.ents:
    print(ent.text, ent.label_)


Apple ORG
UK LOC
